In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import numdifftools as nd

In [53]:
r = np.random.RandomState(42)
d = 10
A = r.random((d,d))
A = (A.T)@A + np.identity(d)
b = r.random((d,))
c = r.random()

def f(x):
    return 1/2*A@x@x - b@x + c + (np.sin(b@x))**3

def opt_step(x1, x2, eps):
    left = x1
    right = x2
    alpha = (1 + np.sqrt(5))/2
    x1 = right - (right - left)/alpha
    x2 = left + (right - left)/alpha
    f1 = f(x + x1*p)
    f2 = f(x + x2*p)
    while right - left > eps:
        if f1 > f2:
            left = x1
            x1 = x2
            x2 = left + (right - left)/alpha
            f1 = f2
            f2 = f(x + x2*p)
        else:
            right = x2
            x2 = x1
            x1 = right - (right - left)/alpha
            f2 = f1
            f1 = f(x + x1*p)
        step = (left + right)/2
    return step

x = np.zeros(d)
g = nd.Gradient(f)(x)
B = np.identity(d)
p = -B@g
gamma = 1

res_sp = minimize(f, x)
sp_min = res_sp.fun

lim = 30

x1, x2, eps = 0, 2, 0.00000001

for _ in range(lim):
    x_old = x
    g_old = g
    x = x + gamma*p
    g = nd.Gradient(f)(x)
    s = g - g_old
    y = x - x_old
    B = B + np.outer(y - B@s,y - B@s)/np.inner(y - B@s,s)
    p = -B@g
    gamma = opt_step(x1, x2, eps)

print('obtained f: ', f(x), 'scipy_solution: ', sp_min)

obtained f:  0.08178055948814872 scipy_solution:  0.08178055950899162
